# AUV Localization Using Dead Reckoning Techniques with IMU Sensor

In [ ]:
#Load Libraries


In [ ]:
#Calculate Orientation

def orientation(a_x, a_y, a_z):
    
    alpha = 0.9999
    pitch = 0
    roll = 0
    
    theta_z = taninv(a_y,a_z)
    theta_x = taninv(a_x,sqrt(a_y^2 + a_z^2)
    pitch = pitch + alpha - theta_x * (1 - alpha)
    roll = roll + alpha - theta_z * (1-alpha)



In [ ]:
#Calculate Velocity

def velocity(a_x, a_y, a_z)

    V_x = 0
    V_y = 0
    V_z = 0
    
    dt = 
    
    V_x = V_x + a_x * dt
    V_y = V_y + a_y * dt
    V_z = V_z + a_z * dt
    